# ETL Report for Census Annual Business Survey
## Introduction
Every year The United States Census Bureau conducts their [Annual Business Survey (ABS)](https://www.census.gov/programs-surveys/abs/about.html) electronically. It provides information on selected economic and demographic characteristics for businesses and business owners by sex, ethnicity, race, and veteran status.

In this project, we use the ABS to learn more about demographics in The United States' businesses.
### Data Sources
All of our data was sourced from The United States Census Bureau [2019 Annual Business Survey](https://www.census.gov/data/developers/data-sets/abs.2019.html) by calling their APIs:
* [Company Summary](https://api.census.gov/data/2018/abscs.html)
* [Characteristics of Businesses](https://api.census.gov/data/2018/abscb.html)
* [Characteristics of Business Owners](https://api.census.gov/data/2018/abscbo.html)
* [Technology Characteristics of Businesses](https://api.census.gov/data/2018/abstcb.html)

Below you will find an outline for the Extract, Transform, and Load (ETL) process for the data.

## Extract
1. Import the `api_key` from the config file in order to run API requests
2. Set up API requests with appropriate links, formatting the `api_key`
3. Call the API for each request and store the outcome as a DataFrame using `API2DataFrame`

In [1]:
#import relevant libraries
import requests
from config import api_key
import json
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
#URLs
business_owner_url = f'https://api.census.gov/data/2018/abscbo?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F&for=us:*&for=QDESC_LABEL=ACQBUS&key={api_key}'

business_characteristics_url = f'https://api.census.gov/data/2018/abscb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F&for=us:*&for=QDESC_LABEL=SPOUSES&key={api_key}'

company_summary_url = f'https://api.census.gov/data/2018/abscs?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F&for=us:*&key={api_key}'

state_company_summary_url = f'https://api.census.gov/data/2017/abscs?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F&for=state:*&key={api_key}'

tech_characteristics_url = f'https://api.census.gov/data/2018/abstcb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,NSFSZFI,NSFSZFI_LABEL,FACTORS_P,FACTORS_P_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F&for=us:*&key={api_key}'

In [3]:
def API2DataFrame(url):
    '''
    url: str
    
    takes in url for API and returns a pandas DataFrame
    '''
    #Gather data from the API
    response = requests.get(url)
    #turn into json
    response = response.json()
    #Load data into DataFrame
    df = pd.DataFrame(data = response[1:], columns = response[0])
    
    return df

Let's begin by calling from the API and loading the data into a `pandas` DataFrame

In [4]:
#DataFrames
business_owner_characteristics_raw = API2DataFrame(business_owner_url)

In [5]:

business_characteristics_raw = API2DataFrame(business_characteristics_url)

In [6]:

company_summary_raw = API2DataFrame(company_summary_url)

In [7]:
#uncomment to load state data -- note: it is a lengthy API call
#state_company_summary_raw = API2DataFrame(state_company_summary_url)

In [8]:
tech_characteristics_raw = API2DataFrame(tech_characteristics_url)

## Transform
1. Remove unwanted data columns from the DataFrame (different for each)
2. Drop columns and rows where all entries are NA
3. Remove records where there is a 'withheld' flag (see D flag in [ABS Documentation](https://www2.census.gov/programs-surveys/abs/technical-documentation/api/ABS_API_CBO-1-26-2021.pdf))
4. After filtering flags appropriately, drop flag columns

These steps can all be done by calling `cleanDataFrame()` and feeding it:
* `dataframe` : a `pandas` DataFrame (e.g. those pre-loaded in the Extract step above)
* `drop_columns` : a list of columns to be dropped from the DataFrame
* `filter_columns` : a list of columns to check for flags and filter
* `rename_cols` : a dictionary of the columns to be renamed along with their new name

The function will then return a cleaned DataFrame

In [9]:
def cleanDataFrame(dataframe, drop_columns = [], filter_columns = [], rename_cols = {}):
    '''
    Takes in a pandas dataframe and drops unwanted columns and cleans data according to flag/filters
    
    Parameters
    ----------
    dataframe: a pandas DataFrame
    
    drop_columns: list of columns to be dropped from dataframe
    
    filter_columns: list of columns that act as filter/flags for the record
    
    Returns
    --------
    dataframe : the cleaned dataset as a pandas dataframe
    
    '''
    
    #Remove unwanted columns
    dataframe = dataframe.drop(columns = drop_columns)
    
    #Remove all columns and rows that have all null values
    dataframe.dropna(axis='columns', how='all',inplace=True)
    dataframe.dropna(axis='rows', how='all',inplace=True)
    
    #Remove all records where there is a D flag
    for column in filter_columns:
        #Remove records for one flag at a time
        dataframe.drop(dataframe.loc[dataframe[column] == 'D'].index, inplace=True)
        
    #After filtering flags, we can remove flag columns
    dataframe.drop(columns = filter_columns, inplace = True)
    
    #Rename the columns to something more intutitive
    dataframe.rename(columns = rename_cols, inplace=True)
    
    #Change numeric columns to int or float
    for col in dataframe.columns:
        if ('Percentage' or 'Pay' or 'Revenue') in col:
            dataframe[col] = dataframe[col].astype(float)
        elif ('Number' or 'Size') in col:
            dataframe[col] = dataframe[col].astype(int)
    
    
    
    return dataframe
    

## Business Owner Characteristics

In [10]:
business_owner_characteristics_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104622 entries, 0 to 104621
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   GEO_ID            104622 non-null  object
 1   NAME              104622 non-null  object
 2   NAICS2017         104622 non-null  object
 3   NAICS2017_LABEL   104622 non-null  object
 4   OWNER_SEX         104622 non-null  object
 5   OWNER_SEX_LABEL   104622 non-null  object
 6   OWNER_ETH         104622 non-null  object
 7   OWNER_ETH_LABEL   104622 non-null  object
 8   OWNER_RACE        104622 non-null  object
 9   OWNER_RACE_LABEL  104622 non-null  object
 10  OWNER_VET         104622 non-null  object
 11  OWNER_VET_LABEL   104622 non-null  object
 12  QDESC             104622 non-null  object
 13  QDESC_LABEL       104622 non-null  object
 14  OWNCHAR           104622 non-null  object
 15  OWNCHAR_LABEL     104622 non-null  object
 16  YEAR              104622 non-null  obj

You may notice that some of these columns have codes and are repeated with labels. Let's go ahead and remove the columns that do not add to the data/are redundant

In [11]:
# For each data frame, we'll want to drop specific columns
# Business owner characteristics: GEO_ID, NAICS2017, OWNER_SEX, OWNER_ETH, OWNER_RACE, OWNER_VET, QDESC, OWNCHAR, YEAR, state
drop_columns = ['GEO_ID', 'NAICS2017', 'OWNER_SEX', 'OWNER_ETH', 'OWNER_RACE',
                'OWNER_VET', 'QDESC', 'OWNCHAR', 'YEAR', 'OWNPDEMP_PCT_S', 'OWNPDEMP_S', 'us']

flag_columns = ['OWNPDEMP_F', 'OWNPDEMP_PCT_F', 'OWNPDEMP_S_F', 'OWNPDEMP_PCT_S_F']

renaming = {
    'NAME' : 'Region',
    'NAICS2017_LABEL' : 'IndustrySector',
    'OWNER_SEX_LABEL' : 'OwnerSex',
    'OWNER_ETH_LABEL' : 'OwnerEthnicity',
    'OWNER_RACE_LABEL' : 'OwnerRace',
    'OWNER_VET_LABEL' : 'VeteranStatus',
    'QDESC_LABEL' : 'QuestionDescription',
    'OWNCHAR_LABEL' : 'OwnerResp',
    'OWNPDEMP' : 'NumberOfOwners',
    'OWNPDEMP_PCT' : 'PercentageOfOwners',
}

business_owner_characteristics = cleanDataFrame(
    business_owner_characteristics_raw, drop_columns = drop_columns,
    filter_columns = flag_columns, rename_cols = renaming
)

In [12]:
business_owner_characteristics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92564 entries, 0 to 104621
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Region               92564 non-null  object 
 1   IndustrySector       92564 non-null  object 
 2   OwnerSex             92564 non-null  object 
 3   OwnerEthnicity       92564 non-null  object 
 4   OwnerRace            92564 non-null  object 
 5   VeteranStatus        92564 non-null  object 
 6   QuestionDescription  92564 non-null  object 
 7   OwnerResp            92564 non-null  object 
 8   NumberOfOwners       92564 non-null  int32  
 9   PercentageOfOwners   92564 non-null  float64
dtypes: float64(1), int32(1), object(8)
memory usage: 7.4+ MB


## Business Characteristics

In [13]:
business_characteristics_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48737 entries, 0 to 48736
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   GEO_ID             48737 non-null  object
 1   NAME               48737 non-null  object
 2   NAICS2017          48737 non-null  object
 3   NAICS2017_LABEL    48737 non-null  object
 4   SEX                48737 non-null  object
 5   SEX_LABEL          48737 non-null  object
 6   ETH_GROUP          48737 non-null  object
 7   ETH_GROUP_LABEL    48737 non-null  object
 8   RACE_GROUP         48737 non-null  object
 9   RACE_GROUP_LABEL   48737 non-null  object
 10  VET_GROUP          48737 non-null  object
 11  VET_GROUP_LABEL    48737 non-null  object
 12  QDESC              48737 non-null  object
 13  QDESC_LABEL        48737 non-null  object
 14  BUSCHAR            48737 non-null  object
 15  BUSCHAR_LABEL      48737 non-null  object
 16  YEAR               48737 non-null  objec

You may notice that some of these columns have codes and are repeated with labels. Let's go ahead and remove the columns that do not add to the data/are redundant

In [14]:
# For each data frame, we'll want to drop specific columns
# Business characteristics: GEO_ID, NAICS2017, SEX, ETH_GROUP, RACE_GROUP, VET_GROUP, QDESC, BUSCHAR, YEAR, us
drop_columns = ['GEO_ID', 'NAICS2017', 'SEX', 'ETH_GROUP', 'RACE_GROUP', 'VET_GROUP', 'QDESC', 'BUSCHAR', 'YEAR',
                'FIRMPDEMP_S', 'FIRMPDEMP_PCT_S', 'RCPPDEMP_S', 'RCPPDEMP_PCT_S', 'EMP_S', 'EMP_PCT_S', 'PAYANN_S',
                'PAYANN_PCT_S', 'us']
flag_columns = ['FIRMPDEMP_F', 'FIRMPDEMP_PCT_F', 'RCPPDEMP_F', 'RCPPDEMP_PCT_F', 'EMP_F', 'EMP_PCT_F', 'PAYANN_F',
                'PAYANN_PCT_F', 'FIRMPDEMP_S_F', 'FIRMPDEMP_PCT_S_F', 'RCPPDEMP_S_F', 'RCPPDEMP_PCT_S_F', 'EMP_S_F',
                'EMP_PCT_S_F', 'PAYANN_S_F', 'PAYANN_PCT_S_F']

renaming = {
    'NAME' : 'Region',
    'NAICS2017_LABEL' : 'IndustrySector',
    'SEX_LABEL' : 'Sex',
    'ETH_GROUP_LABEL' : 'Ethnicity',
    'RACE_GROUP_LABEL': 'Race',
    'VET_GROUP_LABEL': 'VeteranStatus',
    'QDESC_LABEL': 'QuestionDescription',
    'BUSCHAR_LABEL': 'SurveyQuestionChoice',
    'FIRMPDEMP': 'NumberOfRespondents',
    'FIRMPDEMP_PCT': 'PercentageOfRespondentsInCategory',
    'RCPPDEMP': 'RevenueOfAllFirmsInCategory',
    'RCPPDEMP_PCT': 'PercentageOfRevenueOfAllFirmsInCategory',
    'EMP': 'NumberOfEmployeesOfAllFirmsInCategory',
    'EMP_PCT': 'PercentageOfEmployees',
    'PAYANN': 'AnnualPay',
    'PAYANN_PCT': 'PercentageOfAnnualPay'
}


business_characteristics = cleanDataFrame(business_characteristics_raw, drop_columns = drop_columns, filter_columns = flag_columns, rename_cols = renaming)

## Company Summary

In [15]:
company_summary_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19458 entries, 0 to 19457
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   GEO_ID            19458 non-null  object
 1   NAME              19458 non-null  object
 2   NAICS2017         19458 non-null  object
 3   NAICS2017_LABEL   19458 non-null  object
 4   SEX               19458 non-null  object
 5   SEX_LABEL         19458 non-null  object
 6   ETH_GROUP         19458 non-null  object
 7   ETH_GROUP_LABEL   19458 non-null  object
 8   RACE_GROUP        19458 non-null  object
 9   RACE_GROUP_LABEL  19458 non-null  object
 10  VET_GROUP         19458 non-null  object
 11  VET_GROUP_LABEL   19458 non-null  object
 12  EMPSZFI           19458 non-null  object
 13  EMPSZFI_LABEL     19458 non-null  object
 14  YEAR              19458 non-null  object
 15  FIRMPDEMP         19458 non-null  object
 16  FIRMPDEMP_F       7516 non-null   object
 17  RCPPDEMP    

In [16]:
# For each data frame, we'll want to drop specific columns
# company summary: GEO_ID, NAICS2017, SEX, ETH_GROUP, RACE_GROUP, VET_GROUP, EMPSZFI, YEAR, us
drop_columns = ['GEO_ID', 'NAICS2017', 'SEX', 'ETH_GROUP', 'RACE_GROUP', 'VET_GROUP', 'EMPSZFI', 'YEAR',
                'FIRMPDEMP_S', 'RCPPDEMP_S', 'EMP_S', 'PAYANN_S', 'us']
flag_columns = ['FIRMPDEMP_F', 'RCPPDEMP_F', 'EMP_F', 'PAYANN_F', 'FIRMPDEMP_S_F', 'RCPPDEMP_S_F', 'EMP_S_F', 'PAYANN_S_F']

renaming = {
    'NAME' : 'Region',
    'NAICS2017_LABEL' : 'IndustrySector',
    'SEX_LABEL' : 'Sex',
    'ETH_GROUP_LABEL' : 'Ethnicity',
    'RACE_GROUP_LABEL': 'Race',
    'VET_GROUP_LABEL' : 'VeteranStatus',
    'EMPSZFI_LABEL' : 'SizeOfFirm',
    'FIRMPDEMP' : 'NumberOfRespondents',
    'RCPPDEMP' : 'TotalRevenueOfFirmsInCategory',
    'EMP' : 'NumberOfEmployeesInCategory',
    'PAYANN' : 'AnnualPayroll',
}

company_summary = cleanDataFrame(company_summary_raw,
                                 drop_columns = drop_columns, filter_columns = flag_columns, rename_cols = renaming)

## State Company Summary

In [17]:
#uncomment if using State data
#state_company_summary_raw.info()

In [18]:
# For each data frame, we'll want to drop specific columns
# state company summary: GEO_ID, NAICS2017, SEX, ETH_GROUP, RACE_GROUP, VET_GROUP, EMPSZFI, YEAR, us

#Can uncomment block of code below if running state data
# drop_columns = ['GEO_ID', 'NAICS2017', 'SEX', 'ETH_GROUP', 'RACE_GROUP', 'VET_GROUP', 'EMPSZFI', 'YEAR',
#                 'FIRMPDEMP_S', 'RCPPDEMP_S', 'EMP_S', 'PAYANN_S', 'state']
# flag_columns = ['FIRMPDEMP_F', 'RCPPDEMP_F', 'EMP_F', 'PAYANN_F', 'FIRMPDEMP_S_F', 'RCPPDEMP_S_F', 'EMP_S_F', 'PAYANN_S_F']

# renaming = {
#     'NAME' : 'Region',
#     'NAICS2017_LABEL' : 'IndustrySector',
#     'SEX_LABEL' : 'Sex',
#     'ETH_GROUP_LABEL' : 'Ethnicity',
#     'RACE_GROUP_LABEL': 'Race',
#     'VET_GROUP_LABEL' : 'VeteranStatus',
#     'EMPSZFI_LABEL' : 'SizeOfFirm',
#     'FIRMPDEMP' : 'NumberOfRespondents',
#     'RCPPDEMP' : 'TotalRevenueOfFirmsInCategory',
#     'EMP' : 'NumberOfEmployeesInCategory',
#     'PAYANN' : 'AnnualPayroll',
# }

# state_company_summary = cleanDataFrame(state_company_summary_raw,
#                                  drop_columns = drop_columns, filter_columns = flag_columns, rename_cols = renaming)

## Technology Characteristics of Businesses

In [19]:
tech_characteristics_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500 entries, 0 to 5499
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   GEO_ID             5500 non-null   object
 1   NAME               5500 non-null   object
 2   NAICS2017          5500 non-null   object
 3   NAICS2017_LABEL    5500 non-null   object
 4   SEX                5500 non-null   object
 5   SEX_LABEL          5500 non-null   object
 6   ETH_GROUP          5500 non-null   object
 7   ETH_GROUP_LABEL    5500 non-null   object
 8   RACE_GROUP         5500 non-null   object
 9   RACE_GROUP_LABEL   5500 non-null   object
 10  VET_GROUP          5500 non-null   object
 11  VET_GROUP_LABEL    5500 non-null   object
 12  NSFSZFI            5500 non-null   object
 13  NSFSZFI_LABEL      5500 non-null   object
 14  FACTORS_P          5500 non-null   object
 15  FACTORS_P_LABEL    5500 non-null   object
 16  YEAR               5500 non-null   object


In [20]:
# For each data frame, we'll want to drop specific columns
# company summary: GEO_ID, NAICS2017, SEX, ETH_GROUP, RACE_GROUP, VET_GROUP, EMPSZFI, YEAR, us
drop_columns = ['GEO_ID', 'NAICS2017', 'SEX', 'ETH_GROUP', 'RACE_GROUP', 'VET_GROUP', 'NSFSZFI', 'YEAR',
                'FIRMPDEMP_S', 'RCPPDEMP_S', 'EMP_S', 'PAYANN_S', 'us']
flag_columns = ['FIRMPDEMP_F', 'RCPPDEMP_F', 'EMP_F', 'PAYANN_F', 'FIRMPDEMP_S_F', 'RCPPDEMP_S_F', 'EMP_S_F', 'PAYANN_S_F']

renaming = {
    'NAME' : 'Region',
    'NAICS2017_LABEL' : 'IndustrySector',
    'SEX_LABEL' : 'Sex',
    'ETH_GROUP_LABEL' : 'Ethnicity',
    'RACE_GROUP_LABEL': 'Race',
    'VET_GROUP_LABEL' : 'VeteranStatus',
    'NSFSZFI_LABEL' : 'SizeOfFirm',
    'FIRMPDEMP' : 'NumberOfRespondents',
    'RCPPDEMP' : 'TotalRevenueOfFirmsInCategory',
    'EMP' : 'NumberOfEmployeesInCategory',
    'PAYANN' : 'AnnualPayroll',
}

tech_characteristics = cleanDataFrame(tech_characteristics_raw,
                                 drop_columns = drop_columns, filter_columns = flag_columns, rename_cols = renaming)

## Load
Load the data to CSV files to be read in by the visualizations notebook

In [21]:
business_owner_characteristics.to_csv(r'Data\business_owner_characteristics.csv')

In [22]:
business_characteristics.to_csv(r'Data\business_characteristics.csv')

In [23]:
company_summary.to_csv(r'Data\company_summary.csv')

In [24]:
#uncomment to save state data
#state_company_summary.to_csv(r'Data\state_company_summary.csv')

In [25]:
tech_characteristics.to_csv(r'Data\tech_characteristics.csv')